In [22]:
import cv2
import numpy as np
import glob

In [2]:
import matplotlib.pyplot as plt

In [3]:
from keras.models import load_model

In [4]:
smoothed_angle = 0

In [33]:
path_images = 'E:/UNI/MCC Proc Imagenes/images/image_03/data/*.png'
#path_images = 'E:/UNI/MCC Proc Imagenes/Trabajo final/gg2/*.jpg'
steering_wheel_path = 'steering_wheel_mercedes.png'

In [34]:
steering_wheel_img = cv2.imread(steering_wheel_path)

In [54]:
model = load_model('model_propuesto.h5')

In [10]:
WIDTH = 100
HEIGHT = 100

In [11]:
def load_image(filename):
    image = cv2.imread(filename)
    return image

In [12]:
def process_image(image):
    processed = cv2.resize(image, dsize=(WIDTH,HEIGHT))
    processed = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)
    processed = np.array(processed)
    processed = processed.astype('float32')
    mean = processed.mean()
    std = processed.std()
    processed = (processed - mean) / std 
    return np.reshape(processed, (-1, WIDTH, HEIGHT, 1))

In [21]:
def predict_angle(filename):
    image = load_image(filename)
    processed = process_image(image)
    angle_predict = model.predict(processed)[0][0] * 180 / 3.14159
    return image, angle_predict


In [55]:
iamges_predicted = []

In [56]:
for filename in glob.glob(path_images):
    image, angle_predict = predict_angle(filename)
    
    rows,cols,channels = steering_wheel_img.shape
    
    rows_image,cols_image,channels_image = image.shape
    
    scale_percent = cols / cols_image  * 10.0 
    
        
    w = int(cols_image * scale_percent / 100)
    h = int(rows_image * scale_percent / 100)
    dim = (w, h)
    
    resized = cv2.resize(steering_wheel_img, dim, interpolation = cv2.INTER_AREA)

    roi = image[0:rows, 0:cols]
    steering_angle = angle_predict
    smoothed_angle += 0.2 * pow(abs((steering_angle - smoothed_angle)), 2.0 / 3.0) * (
            steering_angle - smoothed_angle) / abs(
            steering_angle - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), -smoothed_angle, 1)
    steering_wheel_img_rotate = cv2.warpAffine(steering_wheel_img, M, (cols, rows))


    steering_wheel_img_rotate_gray = cv2.cvtColor(steering_wheel_img_rotate,cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(steering_wheel_img_rotate_gray, 10, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)
    image_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)

    steering_wheel_img_rotate_fg = cv2.bitwise_and(steering_wheel_img_rotate,steering_wheel_img_rotate,mask = mask)
    dst = cv2.add(image_bg,steering_wheel_img_rotate_fg)
    image[0:rows, 0:cols ] = dst
    height, width, layers = image.shape
    size = (width,height)
    iamges_predicted.append(image)    
    

In [57]:
out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(iamges_predicted)):
    out.write(iamges_predicted[i])
out.release()

In [140]:
len(iamges_predicted)

396